In [2]:
import vice
import matplotlib.pyplot as plt
import numpy as np

In [7]:
mw = vice.milkyway(name = "example", N = 2e5, zone_width = 1.0) 
print(mw) 

VICE supplementary data required, downloading now.
You will not need to repeat this process.
vice.milkyway{
    name -----------> example
    n_zones --------> 20
    n_stars --------> 1
    verbose --------> False
    simple ---------> False
    annuli ---------> [0, 1, 2, ... , 18, 19, 20]
    evolution ------> <function milkyway.default_evolution at 0x7fa286328d30>
    mode -----------> ifr
    elements -------> ('fe', 'sr', 'o')
    IMF ------------> kroupa
    mass_loading ---> <function milkyway.default_mass_loading at 0x7fa28632e280>
    dt -------------> 0.01
    bins -----------> [-3, -2.95, -2.9, ... , 0.9, 0.95, 1]
    delay ----------> 0.15
    RIa ------------> plaw
    smoothing ------> 0.0
    tau_ia ---------> 1.5
    m_upper --------> 100.0
    m_lower --------> 0.08
    postMS ---------> 0.1
    Z_solar --------> 0.014
}


In [ ]:
mw.dt = 0.05 
out = mw.run(np.linspace(0, 10, 1001), capture = True)